In [ ]:
!pip install tensorflow==1.14

In [ ]:
!pip install keras-rl

In [ ]:
!pip install gym

In [ ]:
!pip install scikit-learn==0.19.2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def get_labels_only(obj, one_hot_encoded=False):
  output_test_data = []

  for index, obj in enumerate(obj):
    if one_hot_encoded:
      one_hot = np.zeros(64)
      one_hot[obj["label"]] = 1
    else:
      one_hot = obj['label']

    if index % 100 == 0:
      print(index)
    output_test_data.append(one_hot)

  return  output_test_data

In [ ]:
combined_train_features = np.concatenate([train_features, train_text_features], axis=1)
combined_test_features = np.concatenate([test_features, test_text_features], axis=1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors, LSHForest

In [ ]:
lsh = LSHForest(random_state=0)

In [ ]:
lsh.fit(combined_train_features)

In [ ]:
combined_train_features.shape

In [ ]:
import os, sys, math, csv, datetime, time, pickle
import numpy as np
import pandas as pd
import keras

In [ ]:
import numpy as np
import gym
from gym.wrappers.time_limit import TimeLimit

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

ENV_NAME = 'ImageRetrival'

In [ ]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gym.wrappers.time_limit import TimeLimit
from sklearn.neighbors import DistanceMetric


class ImageRetrieval(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, train_images, train_labels, k, query_images=None, knn=None, clf=None):
        super(ImageRetrieval).__init__()
        self.train_images = train_images
        self.train_labels = train_labels
        if query_images is None:
            self.query_image = self.train_images
        else:
            self.query_image = query_images
        self.k = k
        self.dist = DistanceMetric.get_metric(1-'cosine_similarity')
        if knn is not None:
            self.knn = knn
        else:
            self.knn = KNeighborsClassifier(n_neighbors=k)
            self.knn.fit(self.train_images, self.train_labels)
        if clf is None:
            self.clf = SVC()
            self.clf.fit(self.train_images, self.train_labels)
        else:
            self.clf = clf
        self.p = 0
        self.query_image_index = 0
        self.action_space = spaces.Discrete(3)
        self.image_index = 0
        self.multiple_class_len = None
        self.observation_space = spaces.Box(low=0, high=200, shape=(1, 25088))
        self.result_array = []
        self.top_k_indices = None
        self.episode_length = 10
        self.reset()
        self.zero_action = 0
        self.seen_images = 0

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        done = None
        print(action)
        assert self.action_space.contains(action)
        old_query_image_index = self.query_image_index
        current_observation = self.next_observation

        if self.top_k_indices is None:
            # intial state
            print('top k indices = None')
            distances, top_k_indices = self.knn.kneighbors([current_observation], n_neighbors=self.k)
            top_k_indices = top_k_indices[0]
            distances = distances[0]
            current_class = self.clf.predict([current_observation])
            current_classes = self.clf.predict(self.train_images[top_k_indices])

            if len(set(current_classes)) == 1:
                if current_classes[0] == current_class[0]:
                    done = True
                    # update query_image_idnex for next state
                    self.query_image_index += 1
                    self.seen_images += 1
                    if self.seen_images == self.query_image.shape[0]:
                        self.next_observation = np.zeros_like(current_observation)
                    else:
                        if self.query_image_index >= self.query_image.shape[0]:
                            self.query_image_index = 0
                        self.next_observation = self.query_image[self.query_image_index: self.query_image_index + 1][0]
                else:
                    done = False
            else:
                done = False
            self.reward = sum(distances)
            self.top_k_indices = top_k_indices
        else:
            print('top k indices present')

            current_observation = self.next_observation
            current_obs_class = self.clf.predict(current_observation.reshape((1, -1)))[0]

            current_distances, current_indices = self.knn.kneighbors([current_observation], n_neighbors=self.k)
            current_images = self.train_images[current_indices[0]]
            current_classes = self.clf.predict(current_images)

            print('current_classes = ', current_classes, type(current_classes))
            print('current obs class = ', current_obs_class, type(current_obs_class))

            if len(set(current_classes)) == 1 and current_classes[0] == current_obs_class:
                print('happy days')
                done = True
                distances = current_distances[0]
                self.top_k_indices = current_indices[0]
                self.reward = sum(distances)
                # happy days: move ahead to other query
                self.query_image_index += 1
                self.seen_images += 1
                if self.seen_images == self.query_image.shape[0]:
                    self.next_observation = np.zeros_like(current_observation)
                else:
                    if self.query_image_index >= self.query_image.shape[0]:
                        self.query_image_index = 0
                    self.next_observation = self.query_image[self.query_image_index: self.query_image_index + 1][0]
            else:
                done = False
                if self.multiple_class_len is None:
                    self.multiple_class_len = 0
                else:
                    self.multiple_class_len += 1
                try:
                    print('same features')
                    same_features = current_images[current_classes == current_obs_class][self.multiple_class_len:self.multiple_class_len+1]
                    try_distances, try_top_k_indices = self.knn.kneighbors(same_features, n_neighbors=self.k)
                    try_distances = try_distances[0]
                    try_top_k_indices = try_top_k_indices[0]

                    try_classes = self.clf.predict(self.train_images[try_top_k_indices])

                    self.reward = sum(try_distances)
                    self.top_k_indices = try_top_k_indices

                    if len(set(try_classes)) == 1:
                        if try_classes[0] == current_obs_class:
                            done = True
                            # move ahead
                            self.query_image_index += 1
                            self.seen_images += 1
                            if self.seen_images == self.query_image.shape[0]:
                                self.next_observation = np.zeros_like(current_observation)
                            else:
                                if self.query_image_index >= self.query_image.shape[0]:
                                    self.query_image_index = 0
                                self.next_observation = \
                                self.query_image[self.query_image_index: self.query_image_index + 1][0]
                        else:
                            done = False
                    else:
                        done = False
                except Exception as e:
                    # nomore same classes present
                    p = np.random.random(1)[0]
                    if p > 0.5:
                        print('0.5')
                        distances, indices = self.knn.kneighbors([current_observation], n_neighbors=self.k)
                        distances = distances[0]
                        self.top_k_indices = indices[0]
                        self.reward = sum(distances)
                        self.query_image_index += 1
                        self.seen_images += 1
                        if self.seen_images == self.query_image.shape[0]:
                            self.next_observation = np.zeros_like(current_observation)
                        else:
                            if self.query_image_index >= self.query_image.shape[0]:
                                self.query_image_index = 0
                            self.next_observation = \
                            self.query_image[self.query_image_index: self.query_image_index + 1][0]
                        done = True
                    else:
                        print('exploring indices')
                        indices = np.random.choice(np.arange(self.train_images.shape[0]), self.k)
                        distances = self.dist.pairwise([current_observation], self.train_images[self.top_k_indices])[0]

                        self.top_k_indices = indices
                        self.reward = sum(distances)

                        print(self.reward)

                        if distances < 0.2 * self.k:
                            done = True
                            self.query_image_index += 1
                            self.seen_images += 1
                            if self.seen_images == self.query_image.shape[0]:
                                self.next_observation = np.zeros_like(current_observation)
                            else:
                                if self.query_image_index >= self.query_image.shape[0]:
                                    self.query_image_index = 0
                                self.next_observation = \
                                self.query_image[self.query_image_index: self.query_image_index + 1][0]
                        else:
                            done = False

        print('updated_query image index = ', self.query_image_index)
        return self.next_observation, self.reward, done, dict(zip(list(range(self.k)), self.top_k_indices))

    def reset(self):
        self.top_k_indices = None
        self.current_action_k = 0
        self.reward = 0
        self.query_image_index = np.random.choice(np.arange(self.query_image.shape[0]), 1)[0]
        self.next_observation = self.query_image[self.query_image_index]
        return self.next_observation

    def render(self, mode='human'):
        pass

    def close(self):
        del self.env


In [ ]:
k = 10
env = TimeLimit(ImageRetrieval(combined_train_features, train_labels, k, query_images=None, knn=lsh, clf=clf), 100)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=env.observation_space.shape))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
memory = SequentialMemory(limit=7000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=30,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae', 'acc'])

In [ ]:
history = dqn.fit(env, nb_steps=50, visualize=False, verbose=2)

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plr

In [ ]:
%matplotlib inline

plr.plot(history.history['episode_reward'])

In [ ]:
test_k = 10

In [ ]:
test_env = TimeLimit(ImageRetrieval(combined_train_features, np.array(train_labels), test_k, query_images=combined_test_features, knn=lsh, clf=clf), 100)

In [ ]:
test_env

In [ ]:
train_labels_arr = np.array(train_labels)

In [ ]:
returned_classes = []
actual_classes = []

obs = test_env.reset()
acc = 0
count = 0

features_num = combined_test_features.shape[0]

for i in range(features_num):
  current_index = test_env.query_image_index
  action = dqn.forward(combined_test_features[current_index])
  dones = False
  episode = 0
  print('current_idnex = ', current_index)
  print('correct label = ', test_labels[current_index])
  while not dones:
    obs, rewards, dones, info = test_env.step(action)
    print(info)
    # print(i, rewards, dones, info)
    if dones:
      returned_classes.append(train_labels_arr[list(info.values())[:test_k]])
      actual_classes.append(test_labels[current_index])
      print('done is true = ', returned_classes[-1], test_labels[current_index])
      found_nums = np.count_nonzero(np.array(returned_classes[-1]) == test_labels[current_index])
      acc += (found_nums / test_k)
      # acc += int(index_label == labels[0])
      # print('-----------------------------------------------------')
      count += 1
      # print(count)
      if count > 0 and count % 100 == 0:
          print("Done ", count)
          print("Accuracy so far %g %%" % (acc/count * 100))
      print(test_env.query_image_index)

mean_accuracy = acc/features_num
print("\nThe mean accuracy for top %d images is %g %%" % (test_k, mean_accuracy*100))
print('---------------------------------------------------------------------')

In [ ]:
mean_accuracy

In [ ]:
def get_correction_score(y_true_one_hot, y_pred_probs, is_one_hot=False):
  if is_one_hot:
    y_pred = np.argmax(y_pred_probs, axis=1)
  else:
    y_pred = y_pred_probs

  if is_one_hot:
    y_true = np.argmax(y_true_one_hot, axis=1)
  else:
    y_true = y_true_one_hot

  print(y_pred)
  print(y_true)

  print(confusion_matrix(y_true, y_pred))

  print(classification_report(y_true, y_pred))

  print('accuracy = ',accuracy_score(y_true, y_pred))
  print('precision = ', precision_score(y_true, y_pred, average='weighted'))
  print('recall = ', recall_score(y_true, y_pred,average='weighted'))
  print('f1 score = ', f1_score(y_true, y_pred, average='weighted'))

In [ ]:
returned_classes_c = []

for i in range(len(returned_classes)):
  returned_classes_c.append(returned_classes[i][:test_k])

In [ ]:
np.array(returned_classes_c).shape, np.array(actual_classes).shape

In [ ]:
acc = 0

for i in range(np.array(returned_classes_c).shape[0]):
  found_nums = np.count_nonzero(np.array(returned_classes_c[i]) == actual_classes[i])
  acc += (found_nums / test_k)

In [ ]:
acc / len(returned_classes_c)